# 所需库

In [1]:
from PyQt5 import QtWidgets,QtCore, QtGui
from PyQt5.QtCore import QSize, Qt
from PyQt5.QtWidgets import *
from PyQt5.QtGui import QPicture,QPainter
from PyQt5.QtCore import QPointF,QRectF
import sys
import zmq
import pyqtgraph as pg
import time
from datetime import datetime
import pandas as pd
import numpy as np
import random
import configparser
import os

# 模拟数据流

## 全数据

In [2]:
def get_stock_history_from_yahoo(ticker="NFLX"):
    urlstr = "https://finance.yahoo.com/quote/{0}/history?p={0}".format(ticker) 
    hdata = pd.read_html(urlstr)[0][:-1] 
    hdata = hdata.set_index('Date') 
    hdata.index = pd.to_datetime(hdata.index)
    hdata = hdata.astype('d')
    hdata.columns = ['Open','High','Low','Close', 'AdjClose','Volume'] 
    hdata_reindexed=hdata.reset_index()
    valid_set=hdata_reindexed.sort_values(by='Date').reset_index(drop=True)
    return valid_set

In [3]:
def get_stock_history_from_csv(path="nflx.csv"):
    temp=pd.read_csv(path,index_col=0)
    valid_set=temp.sort_values(by='Date').reset_index(drop=True)
    return valid_set

In [4]:
valid_set1=get_stock_history_from_csv()

In [5]:
int_list=np.arange(6*60*60)
x_list=[]
temp=pd.date_range('9:00',periods=6*60*60,freq='S')
for i in temp:
    temp=(str(i).split()[1].replace(':',''))
    if temp[0]=='0':
        x_list.append(temp[1:])
    else:
        x_list.append(temp[:])

In [6]:
valid_set1['Date']=x_list[:100]

## 模拟数据流

In [7]:
valid_tuple_list1=[]
for i in range(len(valid_set1)):
    valid_tuple_list1.append(tuple(valid_set1.iloc[i]))

因为需要实现 历史数据的再读取工作，因此需要从新的socket 获取全的数据源

对 tuple list 重排序

In [8]:
temp1=valid_tuple_list1[10:50]
temp2=valid_tuple_list1[50:]
valid_tuple_list1[10:50]=temp2
valid_tuple_list1[50:]=temp1

In [9]:
def get_stock_info():
    global valid_tuple_list1
    if len(valid_tuple_list1)>0:
        temp=valid_tuple_list1[0]
        valid_tuple_list1=valid_tuple_list1[1:]
        temp=list(temp)
        temp.insert(0,'TEST')
        return temp

In [10]:
valid_tuple_list1[:10]

[('90000', 371.46, 391.56, 370.6, 371.71, 371.71, 10949100.0),
 ('90001', 364.21, 376.77, 356.8, 369.03, 369.03, 11178600.0),
 ('90002', 373.11, 381.36, 364.5, 381.05, 381.05, 6997900.0),
 ('90003', 381.03, 393.52, 367.4, 368.77, 368.77, 8364600.0),
 ('90004', 377.77, 384.01, 370.51, 383.79, 383.79, 5487300.0),
 ('90005', 381.0, 391.4, 368.64, 372.78, 372.78, 8747000.0),
 ('90006', 367.7, 371.31, 356.85, 368.97, 368.97, 8147200.0),
 ('90007', 343.86, 357.47, 341.72, 346.49, 346.49, 7405500.0),
 ('90008', 356.43, 364.54, 347.85, 364.13, 364.13, 7604400.0),
 ('90009', 358.92, 362.98, 344.79, 349.92, 349.92, 6036000.0)]

## 读取config

In [11]:
def read_config():
    config=pd.read_csv("./depend/socket.txt",header=None)
    return list(config[0])

## 随机颜色

In [12]:
def rndColor_list():
    return (random.randint(0, 256), random.randint(0,256), random.randint(0, 256))


# 绘制

## 在界面中的图形类

In [13]:
class DrawRecItem(pg.GraphicsObject):
    def __init__(self,data):
        super().__init__()
        self.data=data
        self.color_list=[(217, 194, 234),
                         (58, 135, 162),
                         (196, 62, 244),
                         (147, 223, 153),
                         (103, 182, 144),
                         (194, 22, 47),
                         (22, 182, 89)]
        self.draw_rect()
        
        
    def draw_rect(self):
        
        self.picture=QPicture()
        p1=QPainter(self.picture)
        
        ### 画k线图 ##########
        
        # 设置画pen 颜色，用来画线
        # 颜色代码使用 RGB值，缩写，都可以
#         p1.setPen(pg.mkPen((0,0,0)))
#         for i in range(len(self.data)):
#             #画一条最大值最小值之间的线
#             p1.drawLine(QPointF(i,self.data[i][3]),QPointF(i,self.data[i][2]))
#             # 设置画刷颜色
#             if self.data[i][1]>self.data[i][4]:
#                 p1.setBrush(pg.mkBrush('g'))
#             else:
#                 p1.setBrush(pg.mkBrush('r'))
#             p1.drawRect(QRectF(i-0.3,self.data[i][1],0.6,self.data[i][4]-self.data[i][1]))
            
        #### 画自定义的线 ####################
#          ## TEST close线
#         p1.setPen(pg.mkPen(255,0,0))
#         for i in range(len(self.data['TEST'])-1):
#             p1.drawLine(QPointF(i,self.data['TEST'][i][4]),QPointF(i+1,self.data['TEST'][i+1][4]))
        
        count=0
        for index in self.data.keys():
            rand_col=self.color_list[count]
            p1.setPen(pg.mkPen(rand_col))
            for i in range(len(self.data[index])-1):
                p1.drawLine(QPointF(i,self.data[index][i][4]),QPointF(i+1,self.data[index][i+1][4]))
            count+=1
#         ## FLOW  close线
#         p1.setPen(pg.mkPen(255,0,0))
#         for i in range(len(self.data['FLOW'])-1):
#             p1.drawLine(QPointF(i,self.data['FLOW'][i][4]),QPointF(i+1,self.data['FLOW'][i+1][4]))
        
#         ## FLML  close线
#         p1.setPen(pg.mkPen(0,255,0))
#         for i in range(len(self.data['FLML'])-1):
#             p1.drawLine(QPointF(i,self.data['FLML'][i][4]),QPointF(i+1,self.data['FLML'][i+1][4]))
            
#         ## ML  close线
#         p1.setPen(pg.mkPen(0,0,255))
#         for i in range(len(self.data['ML'])-1):
#             p1.drawLine(QPointF(i,self.data['ML'][i][4]),QPointF(i+1,self.data['ML'][i+1][4]))
        
    def paint(self,p,*args):
        p.drawPicture(0,0,self.picture)
    def boundingRect(self):
        return QRectF(self.picture.boundingRect())


## 时间轴

In [14]:
class MyAxisItem(pg.AxisItem):
    def __init__(self,ticks,*args,**kwargs):
        pg.AxisItem.__init__(self,*args,**kwargs)
        self.x_values=[x[0] for x in ticks]
        self.x_strings=[x[1] for x in ticks]
    
    def tickStrings(self, values, scale, spacing):
        strings=[]
        for v in values:
            vs=v*scale
            if vs in self.x_values:
                vstr=self.x_strings[np.abs(self.x_values-vs).argmin()]

            else:
                vstr=''
            
            strings.append(vstr)
        return strings

## Update_tab2

In [15]:
#### tab2 线程############################
class Update_tab2(QtCore.QThread):
    requestChange = QtCore.pyqtSignal(str,tuple)

    def __init__(self,config, parent=None):
        super(Update_tab2, self).__init__(parent)
        # 设置工作状态与初始num数值
        self.config=config

    def __del__(self):
        # 线程状态改变与线程终止
        self.working = False
        self.wait()

    def run(self):
        context = zmq.Context()
        sock = context.socket(zmq.SUB)
        for i in self.config:
            sock.setsockopt(zmq.SUBSCRIBE, i.encode())
        
        #新添一个 HIST 表示符
        sock.setsockopt(zmq.SUBSCRIBE, b"HIST")
        sock.setsockopt(zmq.HEARTBEAT_IVL,     5000)
        sock.setsockopt(zmq.HEARTBEAT_TIMEOUT, 3000)

        sock.connect("tcp://192.168.0.32:19007")
#         for i in range(100):
        while True:
#             msg=get_stock_info()
#             time.sleep(1)
            sss=sock.recv()
            msg = sss.decode("ascii").split(",")
            new_data_collection=[]
                
#             if msg[0]=='ML':
#                 print(msg)
            
            lag=msg[0]
            if lag in self.config:
                new_data_collection.append(msg[1])
                for i in msg[2:]:
                    new_data_collection.append(float(i)*10000)
                self.requestChange.emit(lag,tuple(new_data_collection))
            
            if lag=='HIST':
                new_data_collection.append(msg[1])
                new_data_collection.append(msg[2])
                for i in msg[3:]:
                    new_data_collection.append(float(i)*10000)
                self.requestChange.emit(lag,tuple(new_data_collection))
            
#             if msg[0]=='FLOW':
#                 new_data_FLOW.append(msg[1])
#                 for i in msg[2:]:
#                     new_data_FLOW.append(float(i)*10000)
#                 print('FLOW ',new_data_FLOW)

#                 self.requestChange.emit('FLOW',tuple(new_data_FLOW))
#             if msg[0]=='FLML':
#                 new_data_FLML.append(msg[1])
#                 for i in msg[2:]:
#                     new_data_FLML.append(float(i)*10000)
#                 print('FLML ',new_data_FLML)
#                 self.requestChange.emit('FLML',tuple(new_data_FLML))
#             if msg[0]=='ML':
#                 time.sleep(1)
#                 new_data_ML.append(msg[1])
#                 for i in msg[2:]:
#                     new_data_ML.append(float(i)*10000)
#                 print('ML ',new_data_ML)
#                 self.requestChange.emit('ML',tuple(new_data_ML))
                
                
                
#             if msg!= None and msg[0]=='TEST':
#                 #print(tuple(msg[1:]))
#                 self.requestChange.emit('TEST',tuple(msg[1:]))
#                 #self.requestChange.emit('TEST2',('090000',0,0,0))
# #                 #print('ML ',new_data_ML)
                

## 主界面

In [16]:
## 主基类，是 整个GUI的主窗口，内部含有三个子窗口
class Control_sys_Tab(QTabWidget):
    def __init__(self, parent=None):
        
        self.Data={
                  }
        self.color_list = [(193, 210, 240),
                           (58, 135, 162),
                           (196, 62, 244),
                           (147, 223, 153),
                           (103, 182, 144),
                           (194, 22, 47),
                           (22, 182, 89)]
        
        self.time_dict={}
        self.RequestRowKey ={}
        self.OrderRowKey  ={}
        self.BatchRowKey  ={}   #To manage batch row index
        self.ErrorRowKey  ={}   #To manage error row index
        self.BuyBatchValue = {}
        self.SellBatchValue= {}
        self.BatchManagers  ={} # To manage value for each batch
        self.g_CurrRequestRow = 0
        self.g_CurrOrderRow  = 0
        self.g_CurrBatchRow  = 0
        
        # 判断有无历史数据
        self.with_hist=False
        
        self.g_CurrErrorRow  = 0
        super().__init__(parent)
        
#         # 设置sizepolicy
#         self.setSizePolicy(QtWidgets.QSizePolicy(QtWidgets.QSizePolicy.MinimumExpanding,
#                                                  QtWidgets.QSizePolicy.MinimumExpanding))
        
        self.setObjectName("Control_system")
        self.resize(1800, 985)
        self.setWindowTitle("实时监控系统")

        # 创建3个选项卡小控件窗口
        self.tab1 = QWidget()
        self.tab2 = QWidget()
        self.tab3 = QWidget()

        # 将三个选项卡添加到顶层窗口中
        self.addTab(self.tab1, "交易界面")
        self.addTab(self.tab2, "损益界面")
        self.addTab(self.tab3, "风险分析")

        # 记录tab2 中的数据，后续会从其他类中读取数据，并更新和作图
        self.conf=configparser.ConfigParser()
        self.conf.read('./depend/config.txt')
        
        config_tab2=self.conf.get('tab2','tab2_lines').split(",")
        for i in config_tab2:
            self.Data[i]={}
        self.work=Update_tab2(config_tab2)
  
        

        # 每个选项卡自定义的内容
        self.tab1UI()
        self.tab2UI()
        self.tab3UI()

        #self.pushButton.clicked.connect(self.slotStart)
    ################tab1#################################

    def tab1UI(self):
        # 设置主布局
        layout = QHBoxLayout()

        sec_layout = QFormLayout()

        # 创建表格窗口1
        self.tableWidget1 = QtWidgets.QTableWidget()
        self.tableWidget1.setRowCount(5000)
        self.tableWidget1.setColumnCount(11)
        self.tableWidget1.setObjectName("tableWidget")
        self.tableWidget1.setAutoFillBackground(True)
        self.tableWidget1.setHorizontalHeaderLabels(
            ["Acct", "Instrument", "BatchID", "RQID", "Direction", "OrderSize", "TradedVol", "AvgPrice", "Notional",
             "FillRate", "RefPrice"])
        for i in range(0, 11):
            self.tableWidget1.horizontalHeaderItem(i).setTextAlignment(Qt.AlignHCenter)
        # self.tableWidget.item(0, 0).setFont(font)

        # 表格窗口2
        self.tableWidget2 = QtWidgets.QTableWidget()
        self.tableWidget2.setRowCount(20000)
        self.tableWidget2.setColumnCount(14)
        self.tableWidget2.setObjectName("tableWidget")
        self.tableWidget2.setAutoFillBackground(True)
        self.tableWidget2.setHorizontalHeaderLabels(
            ["OrderRef", "RequestID", "PriceType", "Direction", "OffsetFlag", "HedgeFlag", "LimitPrice", "VolOriginal",
             "VolRemain", "VolTraded", "VolConfirmed", "Status", "OrderSysID", "ExchID"])
        for i in range(0, 14):
            self.tableWidget2.horizontalHeaderItem(i).setTextAlignment(Qt.AlignHCenter)

        # 表格窗口3
        self.tableWidget3 = QtWidgets.QTableWidget()
        # self.tableWidget3.setTextAlignment(Qt.AlignHCenter)
        self.tableWidget3.setRowCount(500)
        self.tableWidget3.setColumnCount(6)
        # self.tableWidget3.setStyleSheet('background-repeat:repeat;')  #font color
        # self.tableWidget3.setStyleSheet('color:darkblue;')  #font color
        # self.tableWidget3.setStyleSheet('text-align:center;')
        # self.tableWidget3.setStyleSheet('vertical-align:super;')
        # self.tableWidget3.setStyleSheet('background-color:lightblue')
        # self.tableWidget3.horizontalHeader().setStyleSheet('QHeaderView::section{background:gray}')

        self.tableWidget3.setObjectName("tableWidget")
        self.tableWidget3.setAutoFillBackground(True)
        self.tableWidget3.setHorizontalHeaderLabels(
            ["BatchID", "AcctName", "BuyNotional", "SellNotional", "BuyFillRate", "SellFillRate"])
        for i in range(0, 6):
            self.tableWidget3.horizontalHeaderItem(i).setTextAlignment(Qt.AlignHCenter)
            # self.tableWidget3.
        self.tableWidget3.setColumnWidth(0, 150)
        self.tableWidget3.setColumnWidth(1, 150)
        self.tableWidget3.setColumnWidth(2, 100)
        self.tableWidget3.setColumnWidth(3, 100)
        self.tableWidget3.setColumnWidth(4, 100)
        self.tableWidget3.setColumnWidth(5, 100)
        # self.label = QtWidgets.QLabel(self.centralwidget)
        # self.label.setGeometry(QtCore.QRect(360, 70, 300, 50))
        # self.label.setObjectName("label")
        # self.label.setAutoFillBackground(True)
        # self.label.setAlignment(QtCore.Qt.AlignCenter)
        # self.label.setStyleSheet("border-image:url(images/title.png)")

        self.pushButton = QtWidgets.QPushButton()
        # self.pushButton.setMaximumWidth(100)
        self.pushButton.setObjectName("pushButton")
        self.pushButton.setText("开始运行")

        # 添加表单2进 子布局
        sec_layout.addWidget(self.tableWidget2)

        # 添加表单3进 子布局
        sec_layout.addWidget(self.tableWidget3)

        # 添加 按钮 进 子布局
        sec_layout.addWidget(self.pushButton)

        # 添加表单1 进主布局，子布局进主布局
        layout.addWidget(self.tableWidget1)
        layout.addLayout(sec_layout)

        self.tab1.setLayout(layout)

    #################tab2#################################
    ######################################

    ### 测试线程用代码
    def execute(self):
        # 启动线程
        self.work.start()
        # 线程自定义信号连接的槽函数
        self.work.requestChange.connect(self.display)

    def display(self,biaoji, new_data):
        # 由于自定义信号时自动传递1个字符串参数，所以在这个槽函数中要接受1个参数
        # print(new_data)
        
        # 对于现有的 时间戳，如果读取的标识符为 HIST 则不需要绘图，直接添加数据
        if new_data[1]!='0':
            if biaoji=='HIST':
                self.Data[new_data[0]][new_data[1]]=tuple(new_data[2:])
                
            else:
                #print(new_data)
                self.plotData(biaoji,new_data)




    ########################################
    # 定义一个计时器
#     def timer_start(self):

#         self.timer = QtCore.QTimer(self)
#         self.timer.timeout.connect(self.plotData)
#         self.timer.start(1000)
    #####################################################
    
    
    ################绘图用函数############################
    def print_slot(self,event=None):
        if event is None:
            print("事件为空")
        else:
            
            pos=event[0] #鼠标的位置为event的第一个值
            
            try:
                if self.plt.sceneBoundingRect().contains(pos):

                    # 一个文本项 用来展示十字对应的信息
                    min_len=min(len(self.Data[k]) for k in self.Data.keys())
                    max_len=max(len(self.Data[k]) for k in self.Data.keys())

    #                     print(pos)
                    mousePoint = self.plt.plotItem.vb.mapSceneToView(pos)  # 转换鼠标坐标
                    index = int(mousePoint.x())  # 鼠标所处的X轴坐标
                    pos_y = int(mousePoint.y())  # 鼠标所处的Y轴坐标

                    # 当时间线统一时
                    # 需要重定位，定位到最近的曲线
                    #print('max_len ',min_len)
                    if -1 < index < min_len:
                        #print('index ',index)
                        min_index=min(self.Data, key=lambda k:abs(self.Data[k][index][4]-pos_y))

                        # 在label中写入HTML
                        self.label.setHtml(
                            "<p style='color:black'><strong>数据源：{0}\
                            <p style='color:black'><strong>时间：{1}</strong></p><p style='color:black'>\
                            开盘：{2}</p><p style='color:black'>\
                            收盘：{3}</p><p style='color:black'>\
                            最高价：<span style='color:red;'>{4}</span></p><p style='color:black'>\
                            最低价：<span style='color:green;'>{5}</span></p>".format(
                                min_index,self.Data[min_index][index][0], self.Data[min_index][index][1], self.Data[min_index][index][4],
                                self.Data[min_index][index][2], self.Data[min_index][index][3]))
                        self.label.setPos(mousePoint.x(), mousePoint.y())  # 设置label的位置
                        #print(self.label)

                    # 当时间线不统一时,只显示数据最多的
                    elif min_len<index<max_len:
                        min_index=min(self.Data, key=lambda k:len(self.Data[k]))
                        # 在label中写入HTML
                        self.label.setHtml(
                            "<p style='color:black'><strong>数据源：{0}\
                            <p style='color:black'><strong>时间：{1}</strong></p><p style='color:black'>\
                            开盘：{2}</p><p style='color:black'>\
                            收盘：{3}</p><p style='color:black'>\
                            最高价：<span style='color:red;'>{4}</span></p><p style='color:black'>\
                            最低价：<span style='color:green;'>{5}</span></p>".format(
                                min_index,self.Data[min_index][index][0], self.Data[min_index][index][1], self.Data[min_index][index][4],
                                self.Data[min_index][index][2], self.Data[min_index][index][3]))
                        self.label.setPos(mousePoint.x(), mousePoint.y())  # 设置label的位置

                    ## 将label添加进 plt    
                    self.plt.addItem(self.label)
                    #print(self.plt.listDataItems())
                    # 设置垂直线条和水平线条的位置组成十字光标
                    self.vLine.setPos(mousePoint.x())
                    self.hLine.setPos(mousePoint.y())
            except Exception as e:
                print("error in print_slot")
    
    
    
    ######################################################
    
#     def append_stock_data(self):
#         temp = get_stock_info()
#         if temp != None:
#             self.Data.append(temp)

    def plotData(self,biaoji, new_data):
        try:
            #print(len(self.Data))
    #         print(new_data)
            #添加数据
            if new_data != None and new_data[0]!='0':
    #             try:
                # new data 为 tuple形
                self.Data[biaoji][new_data[0]]=list(new_data)[1:]
    #             except Exception as e:
    #                 print("error in saving data...")

            #画图
            #由于数据图可能会有一个点 传过去所有的历史数据，所以 现在改为dict存储

            #首先需要对 dict进行排序
            # 首先获得所有 biaoji下的 dict keys 的交集，因为x,y 需要长度相同
            temp=list(self.Data.keys())
            min_time_list=self.Data[temp[0]].keys()

            for x in list(self.Data.keys()):
                #min_time_list=list(set(min_time_list).intersection(set(self.Data[x].keys())))
                min_time_list=list(set(min_time_list).union(set(self.Data[x].keys())))

            min_time_list=sorted(min_time_list,key=lambda d: pd.to_datetime(d,format="%H%M%S"))

            x=[]
            time_out=[]
            for time in min_time_list:
                x.append(self.time_dict[time])
    #         if biaoji=='ML':
    #             print(time_out)


            y=[]


    #         #生成绘图时 需要的x轴，y轴
    #         for i in self.Data[biaoji]:
    #             x.append(self.time_dict[i[0]])
    #             y.append(float(i[3]))

            for i in min_time_list:
                y.append(float(self.Data[biaoji][i][3]))

    #         print(x)
    #         if biaoji=='ML':
    #             print(x)

            index=list(self.Data.keys()).index(biaoji)
            #print(len(self.Data[biaoji]))
            color_id=list(self.Data.keys()).index(biaoji)
            #self.plot_line[index].clear()
            self.plot_line[index].setData(x,y,pen=pg.mkPen(self.color_list[color_id],width=3))
            #print(len(self.Data['TEST2']))
        except Exception as e:
            print("error in plotData")

    ###################################################该部分 代码 为实时更新和删除图片，用于显示自定义类###############################
    #         if new_data != None:
    #             self.Data[biaoji].append(new_data)

    #             item = DrawRecItem(self.D

    #             ## 清空layout2 以重新插入图片
    #             for i in range(self.layout2.count()):
    #                 self.layout2.itemAt(i).widget().deleteLater()

    #             ## 由于新添了
    #             max_len=max(len(self.Data[k]) for k in self.Data.keys())
    #             max_index=max(self.Data, key=lambda k:len(self.Data[k]))

    #             index = range(max_len)
    #             time_list = []
    #             for i in index:
    #                 temp = self.Data[max_index][i][0]
    #                 time_list.append(temp)
    #             ticks = [(i, j) for i, j in zip(index, time_list)]
    #             strAxis = MyAxisItem(ticks, orientation="bottom")
    #             self.plt = pg.PlotWidget(axisItems={'bottom': strAxis})
    #             self.plt2 = pg.PlotWidget()


    #             ## 设置背景颜色
    #             self.plt.setBackground((255, 255, 255))

    #             # 将iTem加入到plotwidget控件中
    #             self.plt.addItem(item)

    #             # 将控件添加到pyqt中
    #             self.layout2.addWidget(self.plt,0,0)

    #             # 添加控件2
    #             #self.layout2.addWidget(self.plt2,0,1)            

    #             # 将layout 布局添加到 tab2中
    #             self.tab2.setLayout(self.layout2)

    #             # 设置标签
    #             self.label = pg.TextItem()

    #             self.vLine = pg.InfiniteLine(angle=90, movable=False, )  # 创建一个垂直线条
    #             self.hLine = pg.InfiniteLine(angle=0, movable=False, )  # 创建一个水平线条
    #             self.plt.addItem(self.vLine, ignoreBounds=True)  # 在图形部件中添加垂直线条
    #             self.plt.addItem(self.hLine, ignoreBounds=True)  # 在图形部件中添加水平线条
    #             min_len=min(len(self.Data[k]) for k in self.Data.keys())
    # #             if min_len>1:
    # #                 self.move_slot=pg.SignalProxy(self.plt.scene().sigMouseMoved, rateLimit=60, slot=self.print_slot)
    #####################################################################################################################################            
            

    def tab2UI(self):
        #         self.timer_start()
        self.layout2 = QGridLayout()
        self.tab2.setLayout(self.layout2)
        
         #生成时间轴列表
        int_list=np.arange(6*60*60+1)
        x_list=[]
        temp=pd.date_range('9:00',periods=6*60*60+1,freq='S')
        for i in temp:
            temp_time=str(i).split()[1].replace(':','')
            if temp_time[0]=='0':
                temp_time=temp_time[1:]
            x_list.append(temp_time)
            
        # 生成绘图时需要使用的时间轴
        count=0
        for x in x_list:
            self.time_dict[x]=count
            count+=1
        
        ticks = [(i, j) for i, j in zip(int_list, x_list)]
        strAxis = MyAxisItem(ticks, orientation="bottom")
        
        
        # 创建画图板
        self.plot_plt=pg.PlotWidget(axisItems={'bottom': strAxis})
        self.plot_plt.addLegend()
        self.plot_plt.showGrid(x=True,y=True)
        self.layout2.addWidget(self.plot_plt)
        
        
        len_keys=len(self.Data.keys())
        #print(self.Data.keys())
        self.plot_line=[0]*len_keys
        count=0
        for i in range(len_keys):
            self.plot_line[i]=self.plot_plt.plot([0],[0],pen=self.color_list[i],name=list(self.Data.keys())[i])
        
        ## 设置背景色
        self.plot_plt.setBackground((255, 255, 255))
        ## 定位图片显示时的坐标轴
        self.plot_plt.setYRange(max=30,min=-30)
        self.plot_plt.setXRange(min=0,max=21600)
        
        #self.timer_start()
        self.execute()
        
        

    def tab3UI(self):

        return

# 极端数据流

get_stock_info 是从valid_tuple_list中获取数据的

In [17]:
temp=get_stock_info()
temp

['TEST', '90000', 371.46, 391.56, 370.6, 371.71, 371.71, 10949100.0]

为了检验绘图功能的有效性，需要输入极端值检测

In [18]:
temp_tuple_list=[(str(i),5,5,5,5) for i in range(100)]

In [19]:
temp_tuple_list[:10]

[('0', 5, 5, 5, 5),
 ('1', 5, 5, 5, 5),
 ('2', 5, 5, 5, 5),
 ('3', 5, 5, 5, 5),
 ('4', 5, 5, 5, 5),
 ('5', 5, 5, 5, 5),
 ('6', 5, 5, 5, 5),
 ('7', 5, 5, 5, 5),
 ('8', 5, 5, 5, 5),
 ('9', 5, 5, 5, 5)]

In [20]:
#valid_tuple_list=temp_tuple_list

## 极端数据流 passed!!!!


# Main

In [21]:
if __name__ == "__main__":
    app = QtWidgets.QApplication.instance()
    if app is None:
         app = QtWidgets.QApplication(sys.argv)
    w = Control_sys_Tab()
    w.show()
    sys.exit(app.exec_())

('111016', -2.63, -2.63, -2.63, -2.63)
('111016', 38.96, 38.96, 38.96, 38.96)
('111016', 29.44, 29.44, 29.44, 29.44)
('111032', -3.19, -3.19, -3.19, -3.19)
('111032', 38.21, 38.21, 38.21, 38.21)
('111032', 30.48, 30.48, 30.48, 30.48)
('111048', -2.65, -2.65, -2.65, -2.65)
('111048', 38.95, 38.95, 38.95, 38.95)
('111048', 31.169999999999998, 31.169999999999998, 31.169999999999998, 31.169999999999998)
('111100', -2.79, -2.79, -2.79, -2.79)
('111100', 38.379999999999995, 38.379999999999995, 38.379999999999995, 38.379999999999995)
('111100', 31.169999999999998, 31.169999999999998, 31.169999999999998, 31.169999999999998)
('111116', -2.75, -2.75, -2.75, -2.75)
('111116', 38.080000000000005, 38.080000000000005, 38.080000000000005, 38.080000000000005)
('111116', 32.48, 32.48, 32.48, 32.48)
('111132', -3.33, -3.33, -3.33, -3.33)
('111132', 38.2, 38.2, 38.2, 38.2)
('111132', 31.79, 31.79, 31.79, 31.79)
('111148', -4.17, -4.17, -4.17, -4.17)
('111148', 38.35, 38.35, 38.35, 38.35)
('111148', 31.07

('112332', -1.7100000000000002, -1.7100000000000002, -1.7100000000000002, -1.7100000000000002)
('112332', 42.15, 42.15, 42.15, 42.15)
('112332', 27.75, 27.75, 27.75, 27.75)
('112348', -1.76, -1.76, -1.76, -1.76)
('112348', 42.09, 42.09, 42.09, 42.09)
('112348', 27.650000000000002, 27.650000000000002, 27.650000000000002, 27.650000000000002)
('112400', -1.1900000000000002, -1.1900000000000002, -1.1900000000000002, -1.1900000000000002)
('112400', 43.14, 43.14, 43.14, 43.14)
('112400', 28.470000000000002, 28.470000000000002, 28.470000000000002, 28.470000000000002)
('112416', -2.4699999999999998, -2.4699999999999998, -2.4699999999999998, -2.4699999999999998)
('112416', 42.43, 42.43, 42.43, 42.43)
('112416', 29.650000000000002, 29.650000000000002, 29.650000000000002, 29.650000000000002)
('112432', -2.3699999999999997, -2.3699999999999997, -2.3699999999999997, -2.3699999999999997)
('112432', 41.63, 41.63, 41.63, 41.63)
('112432', 29.14, 29.14, 29.14, 29.14)
('112448', -2.42, -2.42, -2.42, -2.

('130300', 5.58, 5.58, 5.58, 5.58)
('130300', 44.86000000000001, 44.86000000000001, 44.86000000000001, 44.86000000000001)
('130300', 23.23, 23.23, 23.23, 23.23)
('130316', 6.64, 6.64, 6.64, 6.64)
('130316', 45.1, 45.1, 45.1, 45.1)
('130316', 23.28, 23.28, 23.28, 23.28)
('130332', 5.51, 5.51, 5.51, 5.51)
('130332', 45.47, 45.47, 45.47, 45.47)
('130332', 18.98, 18.98, 18.98, 18.98)
('130348', 4.58, 4.58, 4.58, 4.58)
('130348', 44.660000000000004, 44.660000000000004, 44.660000000000004, 44.660000000000004)
('130348', 18.55, 18.55, 18.55, 18.55)
('130400', 2.39, 2.39, 2.39, 2.39)
('130400', 42.24999999999999, 42.24999999999999, 42.24999999999999, 42.24999999999999)
('130400', 18.53, 18.53, 18.53, 18.53)
('130416', 2.54, 2.54, 2.54, 2.54)
('130416', 42.48, 42.48, 42.48, 42.48)
('130416', 17.27, 17.27, 17.27, 17.27)
('130432', 3.37, 3.37, 3.37, 3.37)
('130432', 44.46, 44.46, 44.46, 44.46)
('130432', 18.52, 18.52, 18.52, 18.52)
('130448', 3.07, 3.07, 3.07, 3.07)
('130448', 43.6, 43.6, 43.6, 4

('131632', 14.389999999999999, 14.389999999999999, 14.389999999999999, 14.389999999999999)
('131632', 61.25, 61.25, 61.25, 61.25)
('131632', 18.779999999999998, 18.779999999999998, 18.779999999999998, 18.779999999999998)
('131648', 14.129999999999999, 14.129999999999999, 14.129999999999999, 14.129999999999999)
('131648', 60.55, 60.55, 60.55, 60.55)
('131648', 17.13, 17.13, 17.13, 17.13)
('131700', 15.75, 15.75, 15.75, 15.75)
('131700', 61.61, 61.61, 61.61, 61.61)
('131700', 18.74, 18.74, 18.74, 18.74)
('131716', 16.01, 16.01, 16.01, 16.01)
('131716', 61.269999999999996, 61.269999999999996, 61.269999999999996, 61.269999999999996)
('131716', 19.009999999999998, 19.009999999999998, 19.009999999999998, 19.009999999999998)
('131732', 15.44, 15.44, 15.44, 15.44)
('131732', 61.989999999999995, 61.989999999999995, 61.989999999999995, 61.989999999999995)
('131732', 20.02, 20.02, 20.02, 20.02)
('131748', 15.86, 15.86, 15.86, 15.86)
('131748', 62.99, 62.99, 62.99, 62.99)
('131748', 21.45000000000

('132932', 28.85, 28.85, 28.85, 28.85)
('132932', 71.5, 71.5, 71.5, 71.5)
('132932', 14.58, 14.58, 14.58, 14.58)
('132948', 30.049999999999997, 30.049999999999997, 30.049999999999997, 30.049999999999997)
('132948', 72.06, 72.06, 72.06, 72.06)
('132948', 14.46, 14.46, 14.46, 14.46)
('133000', 28.97, 28.97, 28.97, 28.97)
('133000', 70.89, 70.89, 70.89, 70.89)
('133000', 14.63, 14.63, 14.63, 14.63)
('133016', 26.96, 26.96, 26.96, 26.96)
('133016', 68.92, 68.92, 68.92, 68.92)
('133016', 13.649999999999999, 13.649999999999999, 13.649999999999999, 13.649999999999999)
('133032', 27.590000000000003, 27.590000000000003, 27.590000000000003, 27.590000000000003)
('133032', 69.97, 69.97, 69.97, 69.97)
('133032', 13.5, 13.5, 13.5, 13.5)
('133048', 28.53, 28.53, 28.53, 28.53)
('133048', 69.43, 69.43, 69.43, 69.43)
('133048', 12.6, 12.6, 12.6, 12.6)
('133100', 29.8, 29.8, 29.8, 29.8)
('133100', 69.31, 69.31, 69.31, 69.31)
('133100', 13.25, 13.25, 13.25, 13.25)
('133116', 28.779999999999998, 28.7799999

('134316', 21.89, 21.89, 21.89, 21.89)
('134316', 70.8, 70.8, 70.8, 70.8)
('134316', 4.15, 4.15, 4.15, 4.15)
('134332', 22.91, 22.91, 22.91, 22.91)
('134332', 71.71000000000001, 71.71000000000001, 71.71000000000001, 71.71000000000001)
('134332', 4.9399999999999995, 4.9399999999999995, 4.9399999999999995, 4.9399999999999995)
('134348', 22.509999999999998, 22.509999999999998, 22.509999999999998, 22.509999999999998)
('134348', 71.66, 71.66, 71.66, 71.66)
('134348', 5.199999999999999, 5.199999999999999, 5.199999999999999, 5.199999999999999)
('134400', 23.46, 23.46, 23.46, 23.46)
('134400', 71.36, 71.36, 71.36, 71.36)
('134400', 4.25, 4.25, 4.25, 4.25)
('134416', 23.67, 23.67, 23.67, 23.67)
('134416', 70.42999999999999, 70.42999999999999, 70.42999999999999, 70.42999999999999)
('134416', 4.6000000000000005, 4.6000000000000005, 4.6000000000000005, 4.6000000000000005)


SystemExit: 0

C:\Users\taosu\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
